# Module 3 - Doing much task wow (with OpenCL)

## Setup

### Library Import
Before doing anything else, we need to import [PyOpenCL](https://documen.tician.de/pyopencl/) and [NumPy](http://www.numpy.org/).

In [ ]:
import pyopencl,numpy

### Setting up platforms, devices and context
We're going to setup the devices and context as explicit objects because we might want to interogate their runtime information.

In [ ]:
platforms = pyopencl.get_platforms()
nvidia_device,intel_device = [platform.get_devices()[0] 
                              for platform in platforms]
nvidia_context,intel_context = [pyopencl.Context(devices=[device]) 
                                for device in (nvidia_device,intel_device)]

## Inspecting Device Properties

### Using the runtime API
1. Selecting the properties of interest
2. print out for each device

In [ ]:
name_properties = {
    "Device Name":pyopencl.device_info.NAME,
    "Device Platform":pyopencl.device_info.PLATFORM,
    "Device Type":pyopencl.device_info.TYPE
}

processing_properties = {
    "Available Compute Units": pyopencl.device_info.MAX_COMPUTE_UNITS,
    "Clockrate": pyopencl.device_info.MAX_CLOCK_FREQUENCY
}

memory_properties = {
    "Available Global Memory": pyopencl.device_info.GLOBAL_MEM_SIZE,
    "Available Constant Memory": pyopencl.device_info.MAX_CONSTANT_BUFFER_SIZE,
    "Available Local Memory" : pyopencl.device_info.LOCAL_MEM_SIZE
}

device_types = {
    pyopencl.device_type.CPU:"CPU",
    pyopencl.device_type.GPU:"GPU"
}

In [ ]:
for device in (nvidia_device,intel_device):
    #print out all of the device name properties, except the device type
    for property_name in sorted(name_properties.keys() - {"Device Type"}):
        property_string_args = (property_name,device.get_info(name_properties[property_name]))
        print("%s: %s"%property_string_args)
        
    #look up the device type
    print("Device Types: %s"%device_types[device.get_info(name_properties["Device Type"])])
    
    #print out all of the processing properties
    for property_name in sorted(processing_properties.keys()):
        property_string_args = (property_name,device.get_info(processing_properties[property_name]))
        print("%s: %s"%property_string_args)
    
    #print out all of the memory properties
    for property_name in sorted(memory_properties.keys()):
        property_string_args = (property_name,device.get_info(memory_properties[property_name]))
        print("%s: %s"%property_string_args)
        
    print("\n")

## Using clinfo (external application)
Rather helpfully, Jupypter lets us run command line applications.

In [ ]:
!clinfo

## Task vs Data Parallelism
### Setting up the program
1. Create a program for Vector element-wise multiplication
2. Compile the programs

In [ ]:
program_source = """
kernel void operation(global double *a,
                      global double *b,
                      global double *c)
{
  int gid = get_global_id(0);
  
  double a_temp = a[gid];
  double b_temp = b[gid];
  double result = (b_temp + a_temp)*(b_temp*a_temp) + (b_temp - a_temp)*(b_temp/a_temp);
  
  c[gid] = result*result;
}
"""
nvidia_program_source,intel_program_source = [pyopencl.Program(context,program_source) 
                                              for context in (nvidia_context,intel_context)]

In [ ]:
nvidia_program,intel_program = [program.build()
                                for program in (nvidia_program_source,intel_program_source)]

### Creating the global memory resource
1. Defining source data parameters
2. Creating the source data
3. Creating the memory resources within the context

In [ ]:
M = 100
N = int(8*2**15)
dt = numpy.float64
dt_size = numpy.dtype(dt).itemsize

In [ ]:
a = numpy.random.random((M,N)).astype(dt)+0.01 #make sure it's not zero
b = numpy.random.random((M,N)).astype(dt)*a
c = numpy.empty_like(a)

In [ ]:
def create_buffers(context,a_size,b_size,c_size):
    a_buffer = pyopencl.Buffer(context,
                               flags = pyopencl.mem_flags.READ_ONLY | pyopencl.mem_flags.ALLOC_HOST_PTR, 
                               size=a_size)
    b_buffer = pyopencl.Buffer(context, 
                               flags=pyopencl.mem_flags.READ_ONLY | pyopencl.mem_flags.ALLOC_HOST_PTR, 
                               size=b_size)
    c_buffer = pyopencl.Buffer(context,
                              flags=pyopencl.mem_flags.WRITE_ONLY | pyopencl.mem_flags.ALLOC_HOST_PTR,
                              size=c_size)
    return a_buffer,b_buffer,c_buffer

In [ ]:
nvidia_a_buffer,nvidia_b_buffer,nvidia_c_buffer = create_buffers(nvidia_context,N*dt_size,N*dt_size,N*dt_size)
intel_a_buffer,intel_b_buffer,intel_c_buffer = create_buffers(intel_context,N*dt_size,N*dt_size,N*dt_size)

## Running the program
### Defining the host program
Similar to how we did it in module 2, but now *we* are setting the workgroup size.

In [ ]:
def copyon(queue,a_row,a_buffer,b_row,b_buffer):
    copyon_events = []
        
    copyon_events += [pyopencl.enqueue_copy(queue,
                                            src=a_row,
                                            dest=a_buffer,
                                            is_blocking = False)]
    copyon_events += [pyopencl.enqueue_copy(queue,
                                            src=b_row,
                                            dest=b_buffer,
                                            is_blocking = False)]
    return copyon_events

In [ ]:
def run_kernel(queue,program,work_items,wg_size,a_buffer,b_buffer,c_buffer,copyon_events):
    kernel_event = program.operation(queue,
                                     (work_items,), #global size
                                     wg_size, #local size
                                     a_buffer,b_buffer,c_buffer,
                                     wait_for = copyon_events)
    return [kernel_event]

In [ ]:
def copyoff(queue,c_buffer,c_row,kernel_events):
    c_row = numpy.empty_like(c_row)
    copyoff_event = pyopencl.enqueue_copy(queue,
                                          src = c_buffer,
                                          dest = c_row,
                                          wait_for = kernel_events,
                                          is_blocking = False)
    return [copyoff_event],c_row

In [ ]:
def compute_norm(queue,a,a_buffer,b,b_buffer,c_buffer,program,wgs):
    c = numpy.empty_like(a)
    total = 0.0
    
    if(wgs!=None): wg_size = (int(a.shape[1]/wgs),)
    else: wg_size = None
        
    for i,(a_row,b_row,c_row) in enumerate(zip(a,b,c)):
        #copying data onto device
        copyon_events = copyon(queue,a_row,a_buffer,b_row,b_buffer)
        
        #running program
        kernel_events = run_kernel(queue,program,a_row.shape[0],wg_size,a_buffer,b_buffer,c_buffer,copyon_events)
        
        #copying data off device
        copyoff_event,c[i] = copyoff(queue,c_buffer,c_row,kernel_events)
        
        #since we might as well do something useful while we wait
        if(i>0): total += c[i-1].sum()
            
        #wait for copy-off to finish
        copyoff_event[0].wait()
            
    total += c[-1].sum()
        
    return total**0.5

### Setting up work queue
1. Creating out of order execution queue.
2. Compute norm using as many work groups as there are compute units.
3. Compare to a reference result

In [ ]:
nvidia_oo_queue = pyopencl.CommandQueue(nvidia_context,
                                        properties = pyopencl.command_queue_properties.OUT_OF_ORDER_EXEC_MODE_ENABLE)
intel_oo_queue = pyopencl.CommandQueue(intel_context,
                                       properties = pyopencl.command_queue_properties.OUT_OF_ORDER_EXEC_MODE_ENABLE)

In [ ]:
nvidia_oo_norm = compute_norm(nvidia_oo_queue,
                              a,nvidia_a_buffer,
                              b,nvidia_b_buffer,
                              nvidia_c_buffer,
                              nvidia_program,
                              None
                             )

intel_oo_norm = compute_norm(intel_oo_queue,
                             a,intel_a_buffer,
                             b,intel_b_buffer,
                             intel_c_buffer,
                             intel_program,
                             None
                            )

In [ ]:
reference_result = numpy.linalg.norm((b+a)*(b*a) + (b-a)*(b/a))

In [ ]:
if(numpy.abs(reference_result - nvidia_oo_norm) > 22): raise Exception("nvidia result does not match!")
if(numpy.abs(reference_result - intel_oo_norm) > 22): raise Exception("intel result does not match!")

### Performance Comparison
1. Create a function for benchmarking performance of the kernel (without the memory copy)
2. Benchmark between the 64 ($2^6$) and 4096 ($2^{12}$) workgroups
3. Plot the results

In [ ]:
import time

In [ ]:
def evaluate_program(queue,a,a_buffer,b,b_buffer,c_buffer,
                     program,device,
                     multiple,n=100):
    
    run_time = 0
    for i in range(n):
        #setup for running the kernel
        copyon_events = copyon(queue,a[i],a_buffer,b[i],b_buffer)
        
        cus = device.get_info(pyopencl.device_info.MAX_COMPUTE_UNITS)
        wgs = multiple*cus
        wg_size = (int(a.shape[1]/wgs),)
        
        #run kernel
        start = time.time()
        run_kernel(queue,program,a[i].shape[0],wg_size,a_buffer,b_buffer,c_buffer,copyon_events)
        #compute_norm(queue,a,a_buffer,b,b_buffer,c_buffer,program,multiple*cus)
        stop = time.time()
        
        run_time += stop - start
    
    return run_time/n

In [ ]:
min_wgs = 5
max_wgs = 16

print("wg\tGPU\t\tCPU")
nvidia_times = []
intel_times = []
for t in range(min_wgs,max_wgs):
    nvidia_times += [evaluate_program(nvidia_oo_queue,a,nvidia_a_buffer,b,nvidia_b_buffer,nvidia_c_buffer,
                                      nvidia_program,nvidia_device,
                                      2**t)]
    intel_times += [evaluate_program(intel_oo_queue,a,intel_a_buffer,b,intel_b_buffer,intel_c_buffer,
                                     intel_program,intel_device,
                                     2**t)]
    
    print("%d:\t%.6f\t%.6f"%(2**t,nvidia_times[-1],intel_times[-1]))

In [ ]:
sequential_time = 0
for i in range(100):
    start = time.time()
    result = (b[i]+a[i])*(b[i]*a[i]) + (b[i]-a[i])*(b[i]/a[i])
    end = time.time()
    
    sequential_time += end-start
    
sequential_time = sequential_time/10
print("seq:\t%.6f"%sequential_time)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(2**numpy.arange(min_wgs,max_wgs),intel_times,label="Intel CPU")
plt.plot(2**numpy.arange(min_wgs,max_wgs),nvidia_times,label="NVIDIA GPU")
plt.hlines(sequential_time,color='r',xmin=2**5,xmax=2**15)

plt.xlim(2**min_wgs,2**(max_wgs-1))
plt.xticks(2**numpy.arange(min_wgs,max_wgs))
plt.xscale('log')
plt.yscale('log')
plt.legend(loc='best')
plt.xlabel('Number of Workgroups')
plt.ylabel('Kernel Time (S)')

## Module Challenge
* Perform any BLAS operation, using a mixture of task and data parallelism
* Characterise the change in any of the values

*Hint: Take advantage of multiple indices.*